In [ ]:
# Hopsworks End-to-End Example: Procrastination to Task Completion Predictor
import hopsworks
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

# 1. Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

# 2. Generate dummy data
def generate_data(n_samples=1000):
    data = pd.DataFrame({
        'procrastination_level': np.random.randint(1, 11, n_samples),
        'coffee_cups': np.random.randint(0, 8, n_samples),
        'last_minute_panic': np.random.choice([True, False], n_samples),
        'zodiac_sign': np.random.choice(['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'], n_samples),
        'task_completion': np.random.randint(0, 101, n_samples) / 100  # Target variable
    })
    return data

data = generate_data()

# 3. Create feature group
fg = fs.get_or_create_feature_group(
    name='procrastination_features',
    version=1,
    primary_key=['procrastination_level', 'coffee_cups', 'last_minute_panic', 'zodiac_sign'],
    description='Features for predicting task completion based on procrastination'
)
fg.insert(data)

# 4. Create feature view
fv = fs.get_or_create_feature_view(
    name='procrastination_view',
    version=1,
    description='Feature view for procrastination prediction',
    query=fg.select_all()
)

# 5. Get training data
X, y = fv.get_training_data(target_name='task_completion')
X_train, X_test, y_train, y_test = fv.train_test_split(0.2)

# 6. Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 8. Save model to Hopsworks Model Registry
mr = project.get_model_registry()
model_dir = "procrastination_model"
joblib.dump(model, model_dir + "/model.pkl")

procrastination_model = mr.python.create_model(
    name="procrastination_predictor",
    metrics={"mse": mse},
    model_schema=fv.schema,
    description="Predicts task completion based on procrastination features"
)
procrastination_model.save(model_dir)

# 9. Create model serving endpoint
ms = project.get_model_serving()
predictor = ms.create_predictor(model=procrastination_model)
deployment = predictor.deploy()

# 10. Make a prediction
input_data = {
    'procrastination_level': 7,
    'coffee_cups': 3,
    'last_minute_panic': True,
    'zodiac_sign': 'Virgo'
}
result = deployment.predict(input_data)
print(f"Predicted task completion: {result[0] * 100:.2f}%")

# 11. Clean up (optional)
deployment.delete()
predictor.delete()
procrastination_model.archive()

print("End-to-end example completed successfully!")